In [1]:
!git clone https://github.com/Ashfinn/tomato-leaf.git

Cloning into 'tomato-leaf'...
remote: Enumerating objects: 17749, done.
remote: Total 17749 (delta 0), reused 0 (delta 0), pack-reused 17749 (from 1)
Receiving objects: 100% (17749/17749), 266.73 MiB | 11.26 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Updating files: 100% (16012/16012), done.


In [7]:
!pip uninstall -y torch torchvision torchaudio

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124


In [8]:
!pip install torch torchvision numpy pillow matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201

# MobileNetV2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os

# Set device to CPU
device = torch.device("cuda")

# Define data transformations
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),  # MobileNetV2 expects 224x224
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet stats
])

test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load PlantVillage tomato dataset
data_dir = "tomato-leaf/dataset"  # Update with your dataset path
train_dataset = datasets.ImageFolder(os.path.join(data_dir), transform=train_transforms)
test_dataset = datasets.ImageFolder(os.path.join(data_dir), transform=test_transforms)

# Split dataset (80-20 split)
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)  # Small batch size for CPU
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

# Load pre-trained MobileNetV2
model = models.mobilenet_v2(pretrained=True)

# Modify the classifier for 10 tomato classes
num_classes = 10
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)  # Optimize only classifier for speed

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Train and evaluate
print("Training MobileNetV2 on tomato leaf dataset...")
train_model(model, train_loader, criterion, optimizer, num_epochs=5)
print("\nEvaluating model...")
evaluate_model(model, test_loader)

# Save the model
torch.save(model.state_dict(), "tomato_leaf_classifier.pth")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 158MB/s]


Training MobileNetV2 on tomato leaf dataset...
Epoch 1/5, Loss: 0.6485, Accuracy: 80.46%
Epoch 2/5, Loss: 0.3857, Accuracy: 87.37%
Epoch 3/5, Loss: 0.3291, Accuracy: 89.12%
Epoch 4/5, Loss: 0.3247, Accuracy: 88.93%
Epoch 5/5, Loss: 0.3215, Accuracy: 88.85%

Evaluating model...
Test Accuracy: 92.79%


# Shuffle Net

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# Set device to CPU
device = torch.device("cuda")

# Define data transformations (256x256 input)
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load PlantVillage tomato dataset
data_dir = "tomato-leaf/dataset"  # Your dataset path
train_dataset = datasets.ImageFolder(os.path.join(data_dir), transform=train_transforms)
test_dataset = datasets.ImageFolder(os.path.join(data_dir), transform=test_transforms)

# Split dataset (80-20)
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

# Load pre-trained ShuffleNet v2
model = models.shufflenet_v2_x1_0(pretrained=True)

# Modify classifier with dropout
num_classes = 10
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, num_classes)
)
model = model.to(device)

# Define loss, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

# Validation function
def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(loader), 100 * correct / total

# Training function with early stopping
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=9, patience=2):
    model.train()
    best_loss = float('inf')
    epochs_no_improve = 0
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        val_loss, val_acc = validate(model, test_loader, criterion)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        scheduler.step(val_loss)
        if val_loss < best_loss:
            best_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), "tomato_leaf_shufflenetv2_best.pth")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

# Evaluate function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Train and evaluate
print("Training ShuffleNet v2 on tomato leaf dataset...")
train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=9, patience=2)
print("\nEvaluating model...")
evaluate_model(model, test_loader)

# Save final model
torch.save(model.state_dict(), "tomato_leaf_shufflenetv2.pth")

Training ShuffleNet v2 on tomato leaf dataset...
Epoch 1/9, Train Loss: 1.8480, Train Acc: 48.27%, Val Loss: 1.5223, Val Acc: 69.43%
Epoch 2/9, Train Loss: 1.2729, Train Acc: 74.73%, Val Loss: 1.1066, Val Acc: 76.40%
Epoch 3/9, Train Loss: 0.9499, Train Acc: 80.66%, Val Loss: 0.8625, Val Acc: 82.73%
Epoch 4/9, Train Loss: 0.7555, Train Acc: 84.78%, Val Loss: 0.7067, Val Acc: 84.64%
Epoch 5/9, Train Loss: 0.6232, Train Acc: 87.42%, Val Loss: 0.6024, Val Acc: 86.95%
Epoch 6/9, Train Loss: 0.5373, Train Acc: 88.84%, Val Loss: 0.5273, Val Acc: 88.29%
Epoch 7/9, Train Loss: 0.4700, Train Acc: 90.20%, Val Loss: 0.4713, Val Acc: 89.10%
Epoch 8/9, Train Loss: 0.4202, Train Acc: 91.29%, Val Loss: 0.4342, Val Acc: 89.76%
Epoch 9/9, Train Loss: 0.3859, Train Acc: 91.68%, Val Loss: 0.3992, Val Acc: 89.63%

Evaluating model...
Test Accuracy: 90.54%


# Efficient Net

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
try:
    from efficientnet_pytorch import EfficientNet
except ImportError:
    print("Installing efficientnet-pytorch...")
    os.system("pip install efficientnet-pytorch")
    from efficientnet_pytorch import EfficientNet

# Set device to CPU
device = torch.device("cuda")

# Define data transformations
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
data_dir = "tomato-leaf/dataset"  # Update with your dataset path
train_dataset = datasets.ImageFolder(os.path.join(data_dir), transform=train_transforms)
test_dataset = datasets.ImageFolder(os.path.join(data_dir), transform=test_transforms)

# Split dataset (80-20)
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

# Load pre-trained EfficientNet-B0
model = EfficientNet.from_pretrained("efficientnet-b0")
num_classes = 10
model._fc = nn.Linear(model._fc.in_features, num_classes)
model = model.to(device)

# Define loss, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model._fc.parameters(), lr=0.0005)  # Try 0.0005
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

# Validation function
def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(loader), 100 * correct / total

# Training function with early stopping
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=7, patience=2):
    model.train()
    best_loss = float('inf')
    epochs_no_improve = 0
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        val_loss, val_acc = validate(model, test_loader, criterion)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        scheduler.step(val_loss)
        if val_loss < best_loss:
            best_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), "tomato_leaf_efficientnetb0_best.pth")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

# Evaluate function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Train and evaluate
print("Training EfficientNet-B0 on tomato leaf dataset...")
train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=7, patience=2)
print("\nEvaluating model...")
evaluate_model(model, test_loader)

# Save final model
torch.save(model.state_dict(), "tomato_leaf_efficientnetb0.pth")

Loaded pretrained weights for efficientnet-b0
Training EfficientNet-B0 on tomato leaf dataset...
Epoch 1/7, Train Loss: 0.9887, Train Acc: 74.75%, Val Loss: 0.5050, Val Acc: 88.17%
Epoch 2/7, Train Loss: 0.4087, Train Acc: 89.83%, Val Loss: 0.3453, Val Acc: 90.54%
Epoch 3/7, Train Loss: 0.2959, Train Acc: 92.17%, Val Loss: 0.2716, Val Acc: 92.66%
Epoch 4/7, Train Loss: 0.2411, Train Acc: 93.59%, Val Loss: 0.2361, Val Acc: 93.13%
Epoch 5/7, Train Loss: 0.2050, Train Acc: 94.28%, Val Loss: 0.2112, Val Acc: 94.04%
Epoch 6/7, Train Loss: 0.1845, Train Acc: 94.95%, Val Loss: 0.2003, Val Acc: 94.01%
Epoch 7/7, Train Loss: 0.1626, Train Acc: 95.56%, Val Loss: 0.1840, Val Acc: 94.47%

Evaluating model...
Test Accuracy: 94.26%
